In [1]:
#for laptop usage
cd C:\Users\alfiy\Desktop\BITS\SLAB\SLAB-main\Code

[WinError 3] The system cannot find the path specified: 'C:\\Users\\alfiy\\Desktop\\BITS\\SLAB\\SLAB-main\\Code'
C:\Users\Alfiya\Desktop\SLAB-main\Code\Notebooks


In [1]:
cd C:\Users\Alfiya\Desktop\SLAB-main\Code

C:\Users\Alfiya\Desktop\SLAB-main\Code


In [2]:
import os
#Necessary packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os
import joblib 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import load_model

from supervised_models import logit, xgb_model, mlp
from utils import perf_metric
from VIME.vime_self import vime_self
from hexr_self import hexr_self

# CASE dataset

In [269]:
os.getcwd()

'C:\\Users\\Alfiya\\Desktop\\SLAB-main\\Code'

In [3]:
#load case dataset
df = pd.read_csv("..\Data\CASE_2class.csv")

valence

In [4]:
#one-hot encode class column
ohe = OneHotEncoder()
#Choose either class1 or class2 to select either valence or arousal
df_ohe = pd.DataFrame(ohe.fit_transform(df[['class1']]).toarray())
df = df.join(df_ohe)

df.drop('class1', axis=1, inplace=True)
df.drop('class2', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('valence', axis=1, inplace=True)
df.drop('arousal', axis=1, inplace=True)

In [41]:
df.columns

Index(['ecg', 'bvp', 'gsr', 'rsp', 'skt', 'emg_zygo', 'emg_coru', 'emg_trap',
       0, 1],
      dtype='object')

In [5]:
X = df.loc[:,:'emg_trap']
Y = df.iloc[:,8:]

In [6]:
#Experimental parameters
label_data_rate = 0.1

#Metrics
metric1 = 'acc'
metric2 = 'auc'

## Train-Test split without normalizing

In [7]:
"""
Train-test split
15% :- Test 
10% of 85% = 8.5% :- Labelled dataset
90% of 85% = 76.5% :- Unlabelled dataset
"""
rng_state = np.random.get_state()
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=33) 

#converting to numpy arrays
x_train = x_train.iloc[:, :].values
y_train = y_train.iloc[:, :].values
x_test = x_test.iloc[:,:].values
y_test = y_test.iloc[:,:].values
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1250316, 8), (220644, 8), (1250316, 2), (220644, 2))

## Train-Test split with normalization

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X[['ecg', 'bvp', 'gsr', 'rsp', 'skt', 'emg_zygo', 'emg_coru', 'emg_trap']] = scaler.fit_transform(X[['ecg', 'bvp', 'gsr', 'rsp', 'skt', 'emg_zygo', 'emg_coru', 'emg_trap']])

"""
Train-test split
15% :- Test 
10% of 85% = 8.5% :- Labelled dataset
90% of 85% = 76.5% :- Unlabelled dataset
"""
rng_state = np.random.get_state()
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=33) 

#converting to numpy arrays
x_train = x_train.iloc[:, :].values
y_train = y_train.iloc[:, :].values
x_test = x_test.iloc[:,:].values
y_test = y_test.iloc[:,:].values
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1250316, 8), (220644, 8), (1250316, 2), (220644, 2))

In [8]:
# Divide labeled and unlabeled data
idx = np.random.permutation(len(y_train))

# Label data : Unlabeled data = label_data_rate:(1-label_data_rate)
label_idx = idx[:int(len(idx)*label_data_rate)]
unlab_idx = idx[int(len(idx)*label_data_rate):]

# Unlabeled data
x_unlab = x_train[unlab_idx, :]

# Labeled data
x_train = x_train[label_idx, :] 
y_train = y_train[label_idx, :]

In [14]:
x_unlab.shape, x_train.shape, y_train.shape

((1125285, 8), (125031, 8), (125031, 2))

In [9]:
#parameters to train the encoder
p_m = 0.3
alpha = 2.0

In [255]:
#logistic regression
logistic_acc = []
logistic_auc = []
y_test_hat = logit(x_train, y_train, x_test)
logistic_acc.append(perf_metric(metric1, y_test, y_test_hat))
logistic_auc.append(perf_metric(metric2, y_test, y_test_hat))

In [256]:
logistic_acc

[0.690138866227951]

In [31]:
logistic_auc

[0.5442909873265825]

## XGBoost Model

In [28]:
#For valence
xgboost_valence_acc = []
xgboost_valence_auc = []

y_test_hat = xgb_model(x_train, y_train, x_test)
xgboost_valence_acc.append(perf_metric(metric1, y_test, y_test_hat))
xgboost_valence_auc.append(perf_metric(metric2, y_test, y_test_hat))

In [29]:
xgboost_valence_acc

[0.9152073022606552]

In [23]:
xgboost_valence_auc

[0.9689981682631682]

## MLP

In [10]:
mlp_parameters = dict()
mlp_parameters['hidden_dim'] = 100
mlp_parameters['epochs'] = 100
mlp_parameters['activation'] = 'relu'
mlp_parameters['batch_size'] = 128
mlp_parameters['num_layers'] = 1

In [22]:
mlp_valence_acc = []
mlp_valence_auc = []

for i in range (0, 5):
    y_test_hat = mlp(x_train, y_train, x_test, mlp_parameters)
    mlp_valence_acc.append(perf_metric(metric1, y_test, y_test_hat))
    mlp_valence_auc.append(perf_metric(metric2, y_test, y_test_hat))

6896/6896 [==============================] - 3s 476us/step


In [23]:
mlp_valence_acc

[0.8197095774188285,
 0.8092628850093363,
 0.8076539584126466,
 0.8142800166784504,
 0.8141848407389278]

In [24]:
mlp_valence_auc

[0.8792138035129042,
 0.8615414938145372,
 0.8675861542658312,
 0.8719069967789903,
 0.8709711601016925]

In [25]:
print("Average Valence Accuracy on CASE dataset using MLP Model is: ", np.mean(mlp_valence_acc))

Average Valence Accuracy on CASE dataset using MLP Model is:  0.8130182556516379


## VIME (MSE)

In [10]:
# Train VIME-Self encoder
vime_self_parameters = dict()
vime_self_parameters['batch_size'] = 128
vime_self_parameters['epochs'] = 10
vime_self_encoder = vime_self(x_unlab, p_m, alpha, vime_self_parameters)
    
# Save encoder
if not os.path.exists('Models'):
  os.makedirs('Models')

file_name = 'Models/VIME_valence_encoder1.h5'
    
vime_self_encoder.save(file_name)  

Epoch 1/10
8792/8792 [==============================] - 5s 565us/step - loss: 1235.6862 - mask_loss: 0.7532 - feature_loss: 617.4678
Epoch 2/10
8792/8792 [==============================] - 5s 558us/step - loss: 1235.1461 - mask_loss: 0.6117 - feature_loss: 617.2663
Epoch 3/10
8792/8792 [==============================] - 5s 559us/step - loss: 1235.1449 - mask_loss: 0.6114 - feature_loss: 617.2673
Epoch 4/10
8792/8792 [==============================] - 5s 562us/step - loss: 1235.1484 - mask_loss: 0.6112 - feature_loss: 617.2678
Epoch 5/10
8792/8792 [==============================] - 5s 560us/step - loss: 1235.1477 - mask_loss: 0.6111 - feature_loss: 617.2681
Epoch 6/10
8792/8792 [==============================] - 5s 558us/step - loss: 1235.1454 - mask_loss: 0.6111 - feature_loss: 617.2692
Epoch 7/10
8792/8792 [==============================] - 5s 596us/step - loss: 1235.1492 - mask_loss: 0.6111 - feature_loss: 617.2701
Epoch 8/10
8792/8792 [==============================] - 5s 563us/step

In [13]:
#predict and evaluate vime self encoder
vime_self_encoder = load_model('Models/VIME_valence_encoder1.h5')

# Test VIME-Self
x_train_hat = vime_self_encoder.predict(x_train)
x_test_hat = vime_self_encoder.predict(x_test)

vime_self_acc = []
vime_self_auc = []

mlp_parameters = dict()
mlp_parameters['hidden_dim'] = 100
mlp_parameters['epochs'] = 100
mlp_parameters['activation'] = 'relu'
mlp_parameters['batch_size'] = 128

6896/6896 [==============================] - 3s 375us/step


In [28]:
#experiment to compute accuracy for different number of hidden layers
mlp_parameters['num_layers'] = 5

y_test_hat = mlp(x_train_hat, y_train, x_test_hat, mlp_parameters)
vime_self_acc.append(perf_metric(metric1, y_test, y_test_hat))
vime_self_auc.append(perf_metric(metric2, y_test, y_test_hat))

6896/6896 [==============================] - 4s 562us/step


In [34]:
vime_self_acc

[0.6909954496836533,
 0.6911359474991389,
 0.6908277587425898,
 0.6917568571998333,
 0.6916843421982922]

In [35]:
vime_self_auc

[0.5599444155338404,
 0.5587095839211217,
 0.5599210223458597,
 0.5600654006410932,
 0.5612911840869972]

In [21]:
vime_self_acc

[0.6909954496836533, 0.6911359474991389, 0.6908277587425898]

In [31]:
vime_self_acc.pop(3)

0.6896856474683201

In [33]:
vime_self_auc.pop(3)

0.555187392749746

## SLAB - Valence

In [15]:
# Train SLAB-Self encoder
slab_self_parameters = dict()
slab_self_parameters['batch_size'] = 128
slab_self_parameters['epochs'] = 15
slab_self_encoder = hexr_self(x_unlab, p_m, alpha, slab_self_parameters)
    
# Save encoder
if not os.path.exists('Models'):
  os.makedirs('Models')

file_name = 'Models/SLAB_valence_encoder3.h5'
    
sslam_self_encoder.save(file_name)  

Epoch 1/15
8792/8792 [==============================] - 6s 596us/step - loss: 0.6228 - mask_loss: 0.6121 - feature_loss: 0.0054
Epoch 2/15
8792/8792 [==============================] - 5s 585us/step - loss: 0.6178 - mask_loss: 0.6101 - feature_loss: 0.0039
Epoch 3/15
8792/8792 [==============================] - 5s 583us/step - loss: 0.6154 - mask_loss: 0.6077 - feature_loss: 0.0038
Epoch 4/15
8792/8792 [==============================] - 5s 584us/step - loss: 0.6145 - mask_loss: 0.6068 - feature_loss: 0.0038
Epoch 5/15
8792/8792 [==============================] - 5s 584us/step - loss: 0.6139 - mask_loss: 0.6063 - feature_loss: 0.0038
Epoch 6/15
8792/8792 [==============================] - 5s 581us/step - loss: 0.6135 - mask_loss: 0.6058 - feature_loss: 0.0038
Epoch 7/15
8792/8792 [==============================] - 5s 582us/step - loss: 0.6131 - mask_loss: 0.6054 - feature_loss: 0.0038
Epoch 8/15
8792/8792 [==============================] - 5s 580us/step - loss: 0.6129 - mask_loss: 0.6052

In [16]:
#predict and evaluate sslam self encoder
sslam_self_encoder = load_model('Models/sslam_valence_encoder3.h5')

# Test sslam
x_train_hat = sslam_self_encoder.predict(x_train)
x_test_hat = sslam_self_encoder.predict(x_test)

sslam_self_acc = []
sslam_self_auc = []

mlp_parameters = dict()
mlp_parameters['hidden_dim'] = 100
mlp_parameters['epochs'] = 100
mlp_parameters['activation'] = 'relu'
mlp_parameters['batch_size'] = 128

6896/6896 [==============================] - 3s 366us/step


In [17]:
#experiment to compute accuracy for different number of hidden layers
mlp_parameters['num_layers'] = 5
for i in range (0, 5):
    y_test_hat = mlp(x_train_hat, y_train, x_test_hat, mlp_parameters)
    sslam_self_acc.append(perf_metric(metric1, y_test, y_test_hat))
    sslam_self_auc.append(perf_metric(metric2, y_test, y_test_hat))

6896/6896 [==============================] - 4s 587us/step


In [18]:
np.round(sslam_self_acc, 4)

array([0.8634, 0.8646, 0.8666, 0.8672, 0.8712])

In [19]:
np.round(sslam_self_auc,4)

array([0.9301, 0.9305, 0.9337, 0.9356, 0.9366])

In [20]:
print("Mean of valence accuracies for sslam is ",round(np.mean(sslam_self_acc),4))

# Compute standard deviation
std_dev_acc = np.std(sslam_self_acc)
print("Standard Deviation of Valence Accuracies using sslam: ", round(std_dev_acc,4))

print("Mean of valence accuracies AUC Score for sslam is ",round(np.mean(sslam_self_auc),4))

Mean of valence accuracies for SLAB is  0.8666
Standard Deviation of Valence Accuracies using SLAB:  0.0027
Mean of valence accuracies AUC Score for SLAB is  0.9333


## sslam - Arousal

In [26]:

# Train sslam-Self encoder
sslam_self_parameters = dict()
sslam_self_parameters['batch_size'] = 128
sslam_self_parameters['epochs'] = 15
sslam_encoder_arousal = hexr_self(x_unlab, p_m, alpha, sslam_self_parameters)
    
# Save encoder
if not os.path.exists('Models'):
  os.makedirs('Models')

file_name = 'Models/sslam_arousal_encoder.h5'
    
sslam_encoder_arousal.save(file_name)  

Epoch 1/10
8792/8792 [==============================] - 6s 599us/step - loss: 8.1701 - mask_loss: 0.6146 - feature_loss: 3.7777
Epoch 2/10
8792/8792 [==============================] - 5s 602us/step - loss: 5.1193 - mask_loss: 0.6125 - feature_loss: 2.2534
Epoch 3/10
8792/8792 [==============================] - 5s 598us/step - loss: 4.7070 - mask_loss: 0.6115 - feature_loss: 2.0478
Epoch 4/10
8792/8792 [==============================] - 5s 593us/step - loss: 4.6156 - mask_loss: 0.6115 - feature_loss: 2.0020
Epoch 5/10
8792/8792 [==============================] - 5s 595us/step - loss: 4.5646 - mask_loss: 0.6116 - feature_loss: 1.9765
Epoch 6/10
8792/8792 [==============================] - 5s 594us/step - loss: 4.5279 - mask_loss: 0.6116 - feature_loss: 1.9582
Epoch 7/10
8792/8792 [==============================] - 5s 592us/step - loss: 4.4944 - mask_loss: 0.6115 - feature_loss: 1.9415
Epoch 8/10
8792/8792 [==============================] - 5s 592us/step - loss: 4.4779 - mask_loss: 0.6115

In [27]:
#predict and evaluate sslam self encoder
sslam_encoder_arousal = load_model('Models/sslam_arousal_encoder.h5')

# Test sslam
x_train_hat = sslam_encoder_arousal.predict(x_train)
x_test_hat = sslam_encoder_arousal.predict(x_test)

sslam_arousal_acc = []
sslam_arousal_auc = []

mlp_parameters = dict()
mlp_parameters['hidden_dim'] = 100
mlp_parameters['epochs'] = 100
mlp_parameters['activation'] = 'relu'
mlp_parameters['batch_size'] = 128

6896/6896 [==============================] - 3s 388us/step


In [28]:
#experiment to compute accuracy for different number of hidden layers
mlp_parameters['num_layers'] = 5
for i in range (0, 5):
    y_test_hat = mlp(x_train_hat, y_train, x_test_hat, mlp_parameters)
    sslam_arousal_acc.append(perf_metric(metric1, y_test, y_test_hat))
    sslam_arousal_auc.append(perf_metric(metric2, y_test, y_test_hat))

6896/6896 [==============================] - 4s 566us/step


In [37]:
np.round(sslam_arousal_acc,4)

array([0.8823, 0.8816, 0.8806, 0.8818, 0.8884])

In [33]:
np.round(sslam_arousal_auc, 4)

array([0.945 , 0.9457, 0.9433, 0.9455, 0.9505])

In [35]:
print("Mean of arousal accuracies for sslam is ",round(np.mean(sslam_arousal_acc),4))

print("Mean of arousal accuracies AUC Score for sslam is ",round(np.mean(sslam_arousal_auc), 4))

# Compute standard deviation
std_dev_acc = np.std(sslam_arousal_acc)
print("Standard Deviation of arousal Accuracies using sslam: ", std_dev_acc)

Mean of arousal accuracies for SLAB is  0.8829
Mean of arousal accuracies AUC Score for SLAB is  0.946
Standard Deviation of arousal Accuracies using SLAB:  0.0027667160788093026


## sslam - Valence with He Weight initialization

In [28]:
# Train sslam-Self encoder
sslam_self_parameters = dict()
sslam_self_parameters['batch_size'] = 128
sslam_self_parameters['epochs'] = 10
sslam_self_encoder = hexr_self(x_unlab, p_m, alpha, sslam_self_parameters)
    
# Save encoder
if not os.path.exists('Models'):
  os.makedirs('Models')

file_name = 'Models/sslam_valence_encoder4.h5'
    
sslam_self_encoder.save(file_name)  

Epoch 1/10
8792/8792 [==============================] - 6s 602us/step - loss: 8.5001 - mask_loss: 0.6138 - feature_loss: 3.9432
Epoch 2/10
8792/8792 [==============================] - 5s 604us/step - loss: 5.1739 - mask_loss: 0.6121 - feature_loss: 2.2809
Epoch 3/10
8792/8792 [==============================] - 5s 583us/step - loss: 4.7583 - mask_loss: 0.6121 - feature_loss: 2.0731
Epoch 4/10
8792/8792 [==============================] - 5s 612us/step - loss: 4.6801 - mask_loss: 0.6117 - feature_loss: 2.0342
Epoch 5/10
8792/8792 [==============================] - 5s 590us/step - loss: 4.6459 - mask_loss: 0.6115 - feature_loss: 2.0172
Epoch 6/10
8792/8792 [==============================] - 5s 586us/step - loss: 4.6216 - mask_loss: 0.6115 - feature_loss: 2.0051
Epoch 7/10
8792/8792 [==============================] - 5s 576us/step - loss: 4.5896 - mask_loss: 0.6115 - feature_loss: 1.9891
Epoch 8/10
8792/8792 [==============================] - 5s 578us/step - loss: 4.5485 - mask_loss: 0.6114

In [10]:
#predict and evaluate sslam self encoder
sslam_self_encoder = load_model('Models/sslam_valence_encoder4.h5')

# Test sslam
x_train_hat = sslam_self_encoder.predict(x_train)
x_test_hat = sslam_self_encoder.predict(x_test)

sslam_self_acc = []
sslam_self_auc = []

mlp_parameters = dict()
mlp_parameters['hidden_dim'] = 100
mlp_parameters['epochs'] = 100
mlp_parameters['activation'] = 'relu'
mlp_parameters['batch_size'] = 128

6896/6896 [==============================] - 2s 355us/step


In [11]:
#experiment to compute accuracy for different number of hidden layers
mlp_parameters['num_layers'] = 5
for i in range (0, 5):
    y_test_hat = mlp(x_train_hat, y_train, x_test_hat, mlp_parameters)
    sslam_self_acc.append(perf_metric(metric1, y_test, y_test_hat))
    sslam_self_auc.append(perf_metric(metric2, y_test, y_test_hat))

6896/6896 [==============================] - 4s 592us/step


In [12]:
np.round(sslam_self_acc, 4)

array([0.8978, 0.8947, 0.8998, 0.8896, 0.8958])

In [13]:
np.round(sslam_self_auc,4)

array([0.9554, 0.9525, 0.9566, 0.9504, 0.9545])

In [14]:
print("Mean of valence accuracies for sslam  is ",round(np.mean(sslam_self_acc),4))

# Compute standard deviation
std_dev_acc = np.std(sslam_self_acc)
print("Standard Deviation of Valence Accuracies using sslam: ", round(std_dev_acc,4))

print("Mean of valence accuracies AUC Score for sslam is ",round(np.mean(sslam_self_auc),4))

Mean of valence accuracies for SLAB  is  0.8956
Standard Deviation of Valence Accuracies using SLAB:  0.0034
Mean of valence accuracies AUC Score for SLAB is  0.9539
